In [6]:
import numpy as np
import matplotlib.pyplot as plt
import gzip
from six.moves import cPickle
import sys    

In [7]:
# selecting parameters
# number of convolutional filters to use
nb_filters = 64 

# filter size    
filter_rows = 1
filter_cols = 2 

# convolution kernel size
nb_conv = 3

# subsampling size 
subsample_rows = 1 
subsample_cols_a = 2    
subsample_cols_b = 1

# batch size 
batch_size = 50

# dimension    
latent_dim = 3
intermediate_dim = 128

# epsilon values 
mean=0.
epsilon_std = 1.0      

# dropout 
drop = 0

# epoch, iteration sizes  
nb_epoch = 1  

# saving models: start & end
nb_start = 0
nb_end = 400       
total_epochs = (nb_end-nb_start)*nb_epoch  
print "Total of %i epochs are set to run" % total_epochs  

Total of 400 epochs are set to run


In [8]:
import os, sys
path_data = './X_110000.pkl.gz' 
path_1 = "./fig"
path_2 = "./imgs"
path_3 = "./hist"
path_4 = "./model"
if not os.path.exists(path_1):
    os.mkdir(path_1, 0755);
if not os.path.exists(path_2):
    os.mkdir(path_2, 0755);
if not os.path.exists(path_3):
    os.mkdir(path_3, 0755);
if not os.path.exists(path_4):
    os.mkdir(path_4, 0755);   
print "directories creation compledted or if already exist - then checked"   

directories creation compledted or if already exist - then checked


In [16]:
with gzip.open(path_data, 'rb') as f3:    
    (x_train_raw, y_train_raw), (x_test_raw,y_test_raw), (x_pred_raw,y_pred_raw) = cPickle.load(f3)  
